# Sentinel-1 SLC Burst Mode: Complete Guide

This notebook demonstrates the complete Sentinel-1 SLC Burst capabilities in phidown.

## What are Sentinel-1 Bursts?

Sentinel-1 operates in **Interferometric Wide (IW)** and **Extra-Wide (EW)** modes using TOPSAR (Terrain Observation with Progressive Scans SAR). Each short pulse sequence is a **burst** - the smallest imaging unit in Sentinel-1 SLC data.

- **IW mode**: 3 sub-swaths (IW1, IW2, IW3)
- **EW mode**: 5 sub-swaths (EW1-EW5)

> ⚠️ **Data Availability**: Burst data is available from **August 2, 2024** onwards.

## Features

| Feature | Description |
|---------|-------------|
| **Burst Search** | Search individual bursts with filters (ID, swath, polarization, orbit) |
| **Coverage Calculation** | Automatic AOI coverage percentage for each result |
| **Orbit Optimization** | Find optimal orbit direction and relative orbit number |
| **Temporal Statistics** | Analyze acquisition frequency and gaps |
| **On-demand Download** | Download bursts via CDSE API |

## Quick Reference

| Parameter | Valid Values |
|-----------|-------------|
| `swath_identifier` | IW1, IW2, IW3, EW1-EW5 |
| `polarisation_channels` | VV, VH, HH, HV |
| `orbit_direction` | ASCENDING, DESCENDING |
| `operational_mode` | IW, EW |
| `platform_serial_identifier` | A (Sentinel-1A), B (Sentinel-1B) |

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from phidown import CopernicusDataSearcher
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Define AOI (Northern Italy)
AOI_WKT = '''POLYGON((
    10.5 44.5, 12.5 44.5, 12.5 46.0, 10.5 46.0, 10.5 44.5
))'''

print('✅ Ready!')

✅ Ready!


---
## 1. Basic Burst Search

Enable burst mode with `burst_mode=True`. Coverage is automatically calculated when an AOI is provided.

In [2]:
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    aoi_wkt=AOI_WKT,
    start_date='2024-08-03T00:00:00',
    end_date='2024-08-10T00:00:00',
    polarisation_channels='VV',
    top=20,
    count=True
)

df = searcher.execute_query()
print(f'Found {len(df)} bursts (Total available: {searcher.num_results})')
searcher.display_results()

Found 65 bursts (Total available: 65)


,Id,coverage,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,1865e8da-878a-4358-84da-d853ea1b52f4,0.00,249432,IW1,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:30.281185Z', 'End'..."
1,af00ff76-50a7-4f44-855a-46eefdd77bbd,0.42,249431,IW2,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:28.355111Z', 'End'..."
2,eb2d08dc-3022-4995-a172-6619bbaea2ce,2.08,249431,IW1,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:27.522908Z', 'End'..."
3,55889d1b-4615-459b-a025-568d4a9ff478,0.16,249430,IW3,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:26.674871Z', 'End'..."
4,43ae1b7f-3bb6-45ed-a950-20fe30dcad59,5.81,249430,IW2,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:25.596834Z', 'End'..."
5,cfe28d6a-4708-4101-920f-ba22e8694ca1,3.20,249430,IW1,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:24.764631Z', 'End'..."
6,317746db-118a-4391-bf26-c965e7e085de,2.59,249429,IW3,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:23.916594Z', 'End'..."
7,8d8f0250-7d62-4547-b84d-0f92edc3c0de,7.43,249429,IW2,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:22.838557Z', 'End'..."
8,465712dc-454a-4c25-bad0-2dfe6b98f024,3.48,249429,IW1,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:22.006354Z', 'End'..."
9,537915b2-fdd1-4f73-af1b-d39bf2e8448c,2.74,249428,IW3,S1A_IW_SLC__1SDV_20240809T170720_20240809T1707...,VV,ASCENDING,"{'Start': '2024-08-09T17:07:21.158317Z', 'End'..."


---
## 2. Filtered Burst Search

Combine multiple parameters for targeted searches.

In [3]:
# Search with specific filters
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    aoi_wkt=AOI_WKT,
    swath_identifier='IW2',           # Specific swath
    polarisation_channels='VV',       # Polarization
    orbit_direction='DESCENDING',     # Orbit direction
    operational_mode='IW',            # Mode
    start_date='2024-08-03T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=50,
    count=True
)

df = searcher.execute_query()
print(f'Found {len(df)} bursts matching filters')
searcher.display_results()

Found 22 bursts matching filters


,Id,coverage,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,78cfe8a5-1db4-4a30-83f8-b54d5ff6b257,0.12,359514,IW2,S1A_IW_SLC__1SDV_20240813T052755_20240813T0528...,VV,DESCENDING,"{'Start': '2024-08-13T05:28:07.060494Z', 'End'..."
1,1d367a1c-c59c-4755-95f8-fd795f307364,1.11,359513,IW2,S1A_IW_SLC__1SDV_20240813T052755_20240813T0528...,VV,DESCENDING,"{'Start': '2024-08-13T05:28:04.302217Z', 'End'..."
2,78c3509b-e3ed-411b-a0a0-6e52652d3844,1.36,359512,IW2,S1A_IW_SLC__1SDV_20240813T052755_20240813T0528...,VV,DESCENDING,"{'Start': '2024-08-13T05:28:01.543940Z', 'End'..."
3,cee26cba-69ec-425d-b890-2ecab5829c55,1.63,359511,IW2,S1A_IW_SLC__1SDV_20240813T052755_20240813T0528...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:58.785663Z', 'End'..."
4,575a7114-8759-4756-80aa-0d630689a0a7,1.90,359510,IW2,S1A_IW_SLC__1SDV_20240813T052755_20240813T0528...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:56.027386Z', 'End'..."
5,d8d9c3f6-2ddb-4fa1-8dbd-812a4bd05653,2.18,359509,IW2,S1A_IW_SLC__1SDV_20240813T052730_20240813T0527...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:53.269110Z', 'End'..."
6,9e0f3e79-a1e2-45ea-b3a9-58767478b2de,2.45,359508,IW2,S1A_IW_SLC__1SDV_20240813T052730_20240813T0527...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:50.510833Z', 'End'..."
7,24b5293f-5587-4a92-9a7c-1c5386a2458b,2.73,359507,IW2,S1A_IW_SLC__1SDV_20240813T052730_20240813T0527...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:47.752556Z', 'End'..."
8,036d45a1-2bca-4c65-a055-f664d9abfc94,3.01,359506,IW2,S1A_IW_SLC__1SDV_20240813T052730_20240813T0527...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:44.994279Z', 'End'..."
9,1c7fa9e2-f0b9-4137-b25d-009294eec40e,2.64,359505,IW2,S1A_IW_SLC__1SDV_20240813T052730_20240813T0527...,VV,DESCENDING,"{'Start': '2024-08-13T05:27:42.236002Z', 'End'..."


---
## 3. Search by Burst ID or Parent Product

Find specific bursts or all bursts from a parent SLC product.

In [4]:
# Search by specific Burst ID
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    # aoi_wkt=AOI_WKT,  # AOI required for coverage calculation (can be omitted if not needed)
    burst_id=15804,   # Specific burst ID
    start_date='2024-08-03T00:00:00',
    end_date='2024-08-31T00:00:00',
    top=20,
    count=True
)

df = searcher.execute_query()
print(f'Found {len(df)} acquisitions of Burst ID 15804')
searcher.display_results()

Found 12 acquisitions of Burst ID 15804


,Id,coverage,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,83822e5a-a910-4c44-ad25-08c478ce84b4,None,15804,IW3,S1A_IW_SLC__1SDV_20240826T060719_20240826T0607...,VH,DESCENDING,"{'Start': '2024-08-26T06:07:22.754600Z', 'End'..."
1,88c38dc7-de05-4667-af51-a37650d98ea6,None,15804,IW3,S1A_IW_SLC__1SDV_20240826T060719_20240826T0607...,VV,DESCENDING,"{'Start': '2024-08-26T06:07:22.754600Z', 'End'..."
2,061299bc-77b5-4ce3-8eb9-fd92e0aa47de,None,15804,IW2,S1A_IW_SLC__1SDV_20240826T060719_20240826T0607...,VV,DESCENDING,"{'Start': '2024-08-26T06:07:21.676563Z', 'End'..."
3,3ba111d3-289b-4259-8db4-d7a14c3add55,None,15804,IW2,S1A_IW_SLC__1SDV_20240826T060719_20240826T0607...,VH,DESCENDING,"{'Start': '2024-08-26T06:07:21.676563Z', 'End'..."
4,44ea5ec6-58f5-490f-ab38-99197ce66fa9,None,15804,IW1,S1A_IW_SLC__1SDV_20240826T060719_20240826T0607...,VH,DESCENDING,"{'Start': '2024-08-26T06:07:20.844360Z', 'End'..."
5,5834cba4-8431-4047-afc9-cf59de37959f,None,15804,IW1,S1A_IW_SLC__1SDV_20240826T060719_20240826T0607...,VV,DESCENDING,"{'Start': '2024-08-26T06:07:20.844360Z', 'End'..."
6,03e875f3-319a-4c8f-935b-685b2224721a,None,15804,IW3,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VH,DESCENDING,"{'Start': '2024-08-14T06:07:21.962547Z', 'End'..."
7,c5288339-81c9-4fd5-82cc-8764450b2a06,None,15804,IW3,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VV,DESCENDING,"{'Start': '2024-08-14T06:07:21.962547Z', 'End'..."
8,5d1ec25e-ca40-4f12-bd36-c291903ee40c,None,15804,IW2,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VV,DESCENDING,"{'Start': '2024-08-14T06:07:20.884510Z', 'End'..."
9,f739d767-f2e5-485b-957a-bdbc3421ee8b,None,15804,IW2,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VH,DESCENDING,"{'Start': '2024-08-14T06:07:20.884510Z', 'End'..."


In [5]:
# Get all bursts from a parent product
searcher = CopernicusDataSearcher()

parent_product = 'S1A_IW_SLC__1SDV_20240802T060719_20240802T060746_055030_06B44E_E7CC.SAFE'

searcher.query_by_filter(
    burst_mode=True,
    parent_product_name=parent_product,
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=100
)

df = searcher.execute_query()
print(f'Found {len(df)} VV bursts in parent product')

if len(df) > 0 and 'SwathIdentifier' in df.columns:
    print(f'\nSwath distribution:')
    print(df['SwathIdentifier'].value_counts())

Found 27 VV bursts in parent product

Swath distribution:
SwathIdentifier
IW3    9
IW2    9
IW1    9
Name: count, dtype: int64


---
## 4. Optimal Orbit Finding

Find the best orbit direction and relative orbit number for maximum AOI coverage.

In [6]:
searcher = CopernicusDataSearcher()

orbit_analysis = searcher.find_optimal_orbit(
    aoi_wkt=AOI_WKT,
    start_date='2024-08-03T00:00:00',
    end_date='2024-08-31T00:00:00',
    product_type='SLC'
)

print('=== ORBIT ANALYSIS ===')
print('\n📊 Ascending Orbits:')
for orbit, stats in orbit_analysis['ascending']['orbits'].items():
    print(f'   Orbit {orbit}: {stats["avg_coverage"]:.1f}% avg coverage ({stats["count"]} products)')

print('\n📊 Descending Orbits:')
for orbit, stats in orbit_analysis['descending']['orbits'].items():
    print(f'   Orbit {orbit}: {stats["avg_coverage"]:.1f}% avg coverage ({stats["count"]} products)')

rec = orbit_analysis['recommended']
if rec:
    print(f'\n🎯 RECOMMENDED: {rec["orbit_direction"]} orbit #{rec["relative_orbit"]} ({rec["expected_coverage"]:.1f}% coverage)')

INFO:phidown.search:Analyzing ASCENDING orbits...
INFO:phidown.search:Analyzing DESCENDING orbits...


=== ORBIT ANALYSIS ===

📊 Ascending Orbits:
   Orbit 15: 20.3% avg coverage (4 products)
   Orbit 44: 13.4% avg coverage (6 products)
   Orbit 117: 54.4% avg coverage (4 products)

📊 Descending Orbits:
   Orbit 95: 50.3% avg coverage (4 products)
   Orbit 168: 39.2% avg coverage (4 products)

🎯 RECOMMENDED: ASCENDING orbit #117 (54.4% coverage)


---
## 5. Temporal Statistics

Analyze acquisition frequency and temporal gaps.

In [7]:
# Search for a longer period
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    collection_name='SENTINEL-1',
    product_type='SLC',
    aoi_wkt=AOI_WKT,
    start_date='2024-08-03T00:00:00',
    end_date='2024-10-31T00:00:00',
    top=100,
    count=True
)

df = searcher.execute_query()
stats = searcher.compute_temporal_statistics(df)

if stats:
    print('=== TEMPORAL STATISTICS ===')
    print(f'\n📊 Total acquisitions: {stats["total_acquisitions"]}')
    print(f'📅 Date range: {stats["date_range"]["start"]} to {stats["date_range"]["end"]}')
    print(f'⏱️  Span: {stats["date_range"]["span_days"]} days')
    
    if stats.get('temporal_gaps'):
        gaps = stats['temporal_gaps']
        print(f'\n🔄 Revisit Intervals:')
        print(f'   Min: {gaps["min_days"]:.1f} days')
        print(f'   Max: {gaps["max_days"]:.1f} days')
        print(f'   Mean: {gaps["mean_days"]:.1f} days')
    
    print('\n📆 By Month:')
    for month, count in stats.get('acquisitions_by_month', {}).items():
        print(f'   {month}: {count}')

=== TEMPORAL STATISTICS ===

📊 Total acquisitions: 70
📅 Date range: 2024-08-04T16:58:38.344770+00:00 to 2024-10-27T16:59:04.560152+00:00
⏱️  Span: 84 days

🔄 Revisit Intervals:
   Min: 0.0 days
   Max: 5.0 days
   Mean: 1.2 days

📆 By Month:
   2024-08: 22
   2024-09: 24
   2024-10: 24


/Users/roberto.delprete/Library/CloudStorage/OneDrive-ESA/Desktop/Repos/phidown/phidown/search.py:1578: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  dates.dt.to_period('M').value_counts().sort_index().items()},


---
## 6. Download Bursts

Bursts are generated **on-demand** via the CDSE API (not S3). An access token is required.

In [ ]:
from pathlib import Path
from phidown.downloader import download_burst_on_demand, get_token

# Uncomment and set your credentials:
# access_token = get_token(username='your_username', password='your_password')
# 
# burst_id = df.iloc[0]['Id']
# download_burst_on_demand(
#     burst_id=burst_id,
#     token=access_token,
#     output_dir=Path('./bursts')
# )

print('⚠️ Download code is commented. Set CDSE credentials to run.')

# ====================== API Examples: ======================

# Example 1B: Download API for Burst

To access a specific burst, the only required parameter is the burst ID. In this case, the CDSE API operates differently from standard product retrieval, as burst products are generated on demand rather than retrieved from pre-stored files. Consequently, S3 commands are not applicable. Instead, a POST request is sent to the CDSE service, which dynamically produces and returns the requested burst to the user. 

In this case, a different type of credential is required—an access token—which can be obtained using the standard CDSE user credentials.

[Implementation based on: https://github.com/eu-cdse/notebook-samples/blob/main/geo/bursts_processing_on_demand.ipynb. All credits to CDSE]

In [ ]:
from pathlib import Path
from phidown.downloader import download_burst_on_demand, get_token

In [ ]:
access_token = get_token(username='your_cdse_username',
                        password='your_cdse_password')

In [ ]:
burst_id = df.iloc[0]['Id']

download_burst_on_demand(burst_id=burst_id, token=access_token,
              output_dir=Path('./'))

## Example 2: Burst Search with Spatial Filter (Area of Interest)

You can combine burst mode with spatial filtering using a WKT polygon to define your area of interest.

In [ ]:
"""
Burst Search with AOI

Search for bursts that intersect with a specific geographic area.
The AOI is defined as a WKT (Well-Known Text) polygon.
"""
# Define an area of interest (example: region in central Europe)
aoi_wkt = """POLYGON((12.655118166047592 47.44667197521409, 
                       21.39065656328509 48.347694733853245, 
                       28.334291357162826 41.877123516783655, 
                       17.47086198383573 40.35854475076158, 
                       12.655118166047592 47.44667197521409))"""

searcher = CopernicusDataSearcher()

# Configure search with AOI and temporal filter
searcher.query_by_filter(
    burst_mode=True,
    aoi_wkt=aoi_wkt,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display results
df = searcher.execute_query()
print(f'Found {len(df)} bursts in the specified area')
searcher.display_results(top_n=5)

## Example 3: Search by Specific Burst ID

If you know the Burst ID you're looking for, you can search for it directly.

In [ ]:
"""
Search by Burst ID

Retrieve all acquisitions of a specific burst by its ID.
Burst IDs are integers that uniquely identify a burst location.
"""
searcher = CopernicusDataSearcher()

# Search for a specific burst ID
searcher.query_by_filter(
    burst_mode=True,
    burst_id=15804,  # Example burst ID
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} acquisitions of burst {15804}')
searcher.display_results(top_n=5)

## Example 4: Search by Swath Identifier and Polarization

Filter bursts by specific swath (IW1, IW2, IW3) and polarization channel (VV, VH, HH, HV).

In [ ]:
"""
Filter by Swath and Polarization

Search for bursts from a specific swath with specific polarization.
Swath identifiers: IW1, IW2, IW3 (Interferometric Wide) or EW1-EW5 (Extra Wide)
Polarization channels: VV, VH, HH, HV
"""
searcher = CopernicusDataSearcher()

# Search for IW2 swath with VV polarization
searcher.query_by_filter(
    burst_mode=True,
    swath_identifier='IW2',
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts from swath IW2 with VV polarization')
searcher.display_results(top_n=5)

## Example 5: Search Bursts from a Specific Parent Product

If you know the parent SLC product name, you can retrieve all its constituent bursts.

In [ ]:
"""
Search by Parent Product Name

Retrieve all bursts from a specific parent SLC product.
This is useful when you want to access individual bursts from a known product.
"""
searcher = CopernicusDataSearcher()

# Example parent product name (replace with an actual product name from your results)
parent_product = 'S1A_IW_SLC__1SDV_20240802T060719_20240802T060746_055030_06B44E_E7CC.SAFE'

searcher.query_by_filter(
    burst_mode=True,
    parent_product_name=parent_product,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    polarisation_channels='VV',
    top=1000  # Get all bursts from this product
)


# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts in parent product')
print(f'Parent product: {parent_product}\n')
searcher.display_results(top_n=100)

## Example 6: Filter by Orbit Direction and Relative Orbit Number

Combine orbit parameters with burst search to find specific orbital acquisitions.

In [ ]:
"""
Filter by Orbit Parameters

Search for bursts from a specific orbit direction and relative orbit number.
This is useful for consistent time-series analysis.
"""
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    orbit_direction='DESCENDING',
    relative_orbit_number=8,
    operational_mode='IW',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-10T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts from descending orbit #8')
searcher.display_results(top_n=5)

## Example 7: Advanced Multi-Parameter Burst Search

Combine multiple burst-specific parameters for highly targeted searches.

In [ ]:
"""
Advanced Multi-Parameter Search

Demonstrate combining multiple burst-specific filters for precise queries.
This example searches for a specific burst ID with additional constraints.
"""
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    burst_id=15804,
    swath_identifier='IW2',
    parent_product_type='IW_SLC__1S',
    orbit_direction='DESCENDING',
    relative_orbit_number=8,
    operational_mode='IW',
    polarisation_channels='VV',
    platform_serial_identifier='A',  # Sentinel-1A
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts matching all criteria')
print(f'Total available: {searcher.num_results}')
searcher.display_results(top_n=10)

## Example 8: Analyzing Burst Results

Once you have burst results, you can analyze the data to understand burst distribution, coverage, and other characteristics.

In [ ]:
"""
Analyze Burst Search Results

Perform basic analysis on burst search results to understand:
- Distribution across swaths
- Polarization channels available
- Parent products
- Temporal coverage
"""
searcher = CopernicusDataSearcher()

# Search for bursts with broad criteria
searcher.query_by_filter(
    burst_mode=True,
    orbit_direction='DESCENDING',
    swath_identifier='IW2',
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=100,
    count=True
)

# Execute query
df = searcher.execute_query()

print(f'=== Burst Search Results Analysis ===\n')
print(f'Total bursts found: {len(df)}')
print(f'Total available: {searcher.num_results}\n')

# Analyze burst distribution
if len(df) > 0:
    print(f'=== Burst Characteristics ===')
    
    # Check available columns
    if 'SwathIdentifier' in df.columns:
        print(f'\nSwath Distribution:')
        print(df['SwathIdentifier'].value_counts())
    
    if 'PolarisationChannels' in df.columns:
        print(f'\nPolarization Channels:')
        print(df['PolarisationChannels'].value_counts())
    
    if 'ParentProductName' in df.columns:
        print(f'\nNumber of unique parent products: {df["ParentProductName"].nunique()}')
    
    if 'BurstId' in df.columns:
        print(f'\nNumber of unique burst IDs: {df["BurstId"].nunique()}')
        print(f'Burst ID range: {df["BurstId"].min()} - {df["BurstId"].max()}')
    
    print(f'\n=== Sample Results ===')
    searcher.display_results(top_n=5)